In [1]:
import random

### Задача:  
- Эмулируем стрельбу из лука (функция `hook`).  
- Чтобы варьировать правила стрельбы передаем callback функцию `fn`
- В функции `callback` выводим на экран координаты выстрела и количество попаданий:  
-- Дополнительно подсчитываем расстояние до центра и запоминаем коорднаты выстрела.  
- Во внешней функции, если расстояние до центра меньше `25` считаем это за попадание:  
-- Удаляем координаты успешного выстрела (храним координаты только промахов);  
-- Увеличиваем на 1 счетчик попаданий
- В конце выводим координаты всех промахов и количество попаданий

### Цель:  
- Посмотреть как узменение params внитри функции `callback` и снаружи синхронизировать между собой.

## Global

In [160]:
def hook(fn):
    while True:
        x = random.randint(0, 100)
        y = random.randint(0, 100)
        fn(x, y)
        yield

In [161]:
def callback(x, y):
    global glob_residual, glob_points, glob_hits
    
    print(f'({x=}, {y=}), #Hits: {glob_hits}')
    glob_residual = ( (50-x)**2 + (50-y)**2 )**0.5
    glob_points.append((x, y))

In [163]:
global glob_position, glob_points, glob_hits
glob_residual = 100
glob_points   = []
glob_hits     = 0

pipeline = hook(callback)
for _ in range(10):
    next(pipeline)
    if glob_residual < 25:
        print('Hit', f'{glob_residual:.2f}')
        del glob_points[-1]
        glob_hits += 1
        
print(glob_points)
print(f'#Hits: {glob_hits}')

(x=70, y=78), #Hits: 0
(x=53, y=97), #Hits: 0
(x=39, y=31), #Hits: 0
Hit 21.95
(x=44, y=58), #Hits: 1
Hit 10.00
(x=15, y=95), #Hits: 2
(x=40, y=16), #Hits: 2
(x=44, y=20), #Hits: 2
(x=39, y=64), #Hits: 2
Hit 17.80
(x=74, y=5), #Hits: 3
(x=41, y=58), #Hits: 3
Hit 12.04
[(70, 78), (53, 97), (15, 95), (40, 16), (44, 20), (74, 5)]
#Hits: 4


In [135]:
val = 1
print('global', val)

def foo():
    val = 2
    print('local', val)
    
    def glob():
        global val
        print('glob', val)
        val += 10
        
    def loc():
        nonlocal val
        print('loc', val)
        val += 20
        
    glob()
    loc()
        
    print('local', val)
    
foo()
    
print('global', val)

global 1
local 2
glob 1
loc 2
local 22
global 11


## Param

In [173]:
def hook(fn, param=None):
    while True:
        x = random.randint(0, 100)
        y = random.randint(0, 100)
        fn(x, y, param)
        yield

In [174]:
def callback(x, y, param):
    residual, points, hits = param
    
    print(f'({x=}, {y=}), #Hits: {hits}')
    residual = ( (50-x)**2 + (50-y)**2 )**0.5
    points.append((x, y))

In [175]:
residual = 100
points   = []
hits     = 0

pipeline = hook(callback, param=(residual, points, hits))
for _ in range(10):
    next(pipeline)
    if residual < 25:
        print('Hit', f'{residual:.2f}')
        del points[-1]
        hits = hits + 1
        
print(points)
print(f'#Hits: {hits}')

(x=90, y=10), #Hits: 0
(x=43, y=20), #Hits: 0
(x=44, y=6), #Hits: 0
(x=72, y=52), #Hits: 0
(x=96, y=71), #Hits: 0
(x=43, y=29), #Hits: 0
(x=7, y=93), #Hits: 0
(x=65, y=59), #Hits: 0
(x=37, y=17), #Hits: 0
(x=2, y=56), #Hits: 0
[(90, 10), (43, 20), (44, 6), (72, 52), (96, 71), (43, 29), (7, 93), (65, 59), (37, 17), (2, 56)]
#Hits: 0


In [183]:
class Number:
    def __init__(self, value):
        self.__value = value
        
    def assign(self, value):
        self.__value = value
        return self
    
    def __add__(self, value):
        return Float(self.__value + value)
    
    def __iadd__(self, value):
        self.__value = self.__value + value
        return self
    
    def __lt__(self, value):
        return self.__value < value
        
    def __repr__(self):
        return str(self.__value) if isinstance(self.__value, int) else f'{self.__value:.2f}'

In [184]:
def callback(x, y, param):
    residual, points, hits = param
    
    print(f'({x=}, {y=}), #Hits: {hits}')
    residual.assign( ( (50-x)**2 + (50-y)**2 )**0.5 )
    points.append((x, y))

In [198]:
residual = Number(100)
points   = []
hits     = Number(0)

pipeline = hook(callback, param=(residual, points, hits))
for _ in range(10):
    next(pipeline)
    if residual < 25:
        print('Hit', residual)
        del points[-1]
        # option 1
        hits = hits + 1
#         pipeline = hook(callback, param=(residual, points, hits))
        # option 2 (preferable)
#         hits += 1
        #
        
print(points)
print(f'#Hits: {hits}')

(x=5, y=18), #Hits: 0
(x=63, y=85), #Hits: 0
(x=88, y=83), #Hits: 0
(x=81, y=35), #Hits: 0
(x=82, y=51), #Hits: 0
(x=76, y=67), #Hits: 0
(x=62, y=13), #Hits: 0
(x=81, y=17), #Hits: 0
(x=14, y=54), #Hits: 0
(x=11, y=91), #Hits: 0
[(5, 18), (63, 85), (88, 83), (81, 35), (82, 51), (76, 67), (62, 13), (81, 17), (14, 54), (11, 91)]
#Hits: 0


## Extended callback function

In [ ]:
def hook(fn):
    while True:
        x = random.randint(0, 100)
        y = random.randint(0, 100)
        fn(x, y)
        yield

In [99]:
def callback(x, y, param=0):
    print(f'{x=}, {y=}, {param=}')

In [104]:
def callback_extend(x, y, arg_param, param=0, kwarg_param=1):
    print(f'{x=}, {y=}, {arg_param=}, {param=}, {kwarg_param=}')

In [105]:
callback(1,2)

x=1, y=2, param=0


In [106]:
callback(1,2,3)

x=1, y=2, param=3


In [107]:
callback_extend(1,2,3)

x=1, y=2, arg_param=3, param=0, kwarg_param=1


In [108]:
def callback(x, y, *, param=0):
    print(f'{x=}, {y=}, {param=}')

In [109]:
def callback_extend(x, y, arg_param, *, param=0, kwarg_param=1):
    print(f'{x=}, {y=}, {arg_param=}, {param=}, {kwarg_param=}')

In [110]:
callback(1,2)

x=1, y=2, param=0


In [111]:
callback(1,2,3)

TypeError: callback() takes 2 positional arguments but 3 were given

In [112]:
callback_extend(1,2,3)

x=1, y=2, arg_param=3, param=0, kwarg_param=1


## Decorator

In [214]:
def hook(fn):
    while True:
        x = random.randint(0, 100)
        y = random.randint(0, 100)
        fn(x, y)
        yield

In [215]:
def callback_extend(x, y, residual=100.0, hits=0, points=None): # points=[]): # is it valid?
    print(f'({x=}, {y=}), #Hits: {hits}')
    residual.assign( ( (50-x)**2 + (50-y)**2 )**0.5 )
    points.append((x, y))

In [216]:
def add_extra_kwargs(**extra_kwargs):
    def decorator(fn):
        def inner(*args, **kwargs):
            return fn(*args, **kwargs, **extra_kwargs)
        return inner
    return decorator

In [217]:
residual = Number(100)
points   = []
hits     = Number(0)

callback = add_extra_kwargs(residual=residual, points=points, hits=hits)(callback_extend)
pipeline = hook(callback)
for _ in range(10):
    next(pipeline)
    if residual < 25:
        print('Hit', residual)
        del points[-1]
        # option 1
        hits = hits + 1
        callback = add_extra_kwargs(residual=residual, points=points, hits=hits)(callback_extend)
        pipeline = hook(callback)
        
print(points)
print(f'#Hits: {hits}')

(x=74, y=15), #Hits: 0
(x=2, y=88), #Hits: 0
(x=74, y=81), #Hits: 0
(x=81, y=9), #Hits: 0
(x=7, y=93), #Hits: 0
(x=13, y=32), #Hits: 0
(x=36, y=74), #Hits: 0
(x=30, y=72), #Hits: 0
(x=30, y=24), #Hits: 0
(x=70, y=9), #Hits: 0
[(74, 15), (2, 88), (74, 81), (81, 9), (7, 93), (13, 32), (36, 74), (30, 72), (30, 24), (70, 9)]
#Hits: 0


## Callable Object

In [230]:
class CallbackObject:
    def __init__(self, fn):
        self.__fn = fn
        self.__kwargs = dict()
        
    def update_kwargs(self, **kwargs):
        self.__kwargs.update(kwargs)
        
    def __call__(self, *args, **kwargs):
        return self.__fn(*args, **kwargs, **self.__kwargs)

In [231]:
residual = Number(100)
points   = []
hits     = Number(0)

callback = CallbackObject(callback_extend)
callback.update_kwargs(residual=residual, points=points, hits=hits)
pipeline = hook(callback)
for _ in range(10):
    next(pipeline)
    if residual < 25:
        print('Hit', residual)
        del points[-1]
        # option 1
        hits = hits + 1
        callback.update_kwargs(hits=hits)
        
print(points)
print(f'#Hits: {hits}')

(x=63, y=15), #Hits: 0
(x=1, y=40), #Hits: 0
(x=70, y=8), #Hits: 0
(x=59, y=23), #Hits: 0
(x=40, y=47), #Hits: 0
Hit 10.44
(x=90, y=59), #Hits: 1
(x=36, y=51), #Hits: 1
Hit 14.04
(x=91, y=10), #Hits: 2
(x=2, y=19), #Hits: 2
(x=67, y=26), #Hits: 2
[(63, 15), (1, 40), (70, 8), (59, 23), (90, 59), (91, 10), (2, 19), (67, 26)]
#Hits: 2


---

In [223]:
import os

import cv2
import imutils

import numpy as np

In [224]:
%load_ext watermark
%watermark -v -m -p cv2,imutils,numpy

Python implementation: CPython
Python version       : 3.9.5
IPython version      : 8.7.0

cv2    : 4.6.0
imutils: 0.5.4
numpy  : 1.23.5

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 69 Stepping 1, GenuineIntel
CPU cores   : 4
Architecture: 64bit



In [225]:
def read_image(file_name, margin = 1000):
    # Open image
    img = cv2.imread(file_name)
    if margin:
        img = cv2.copyMakeBorder(img, margin, margin, margin, margin, borderType=cv2.BORDER_CONSTANT, value=(255, 255, 255))

    return img

def resize_image(img, height=1000):
    # Resize the image
    rsz = imutils.resize(img, height=height)

    # Calculate the ratio
    ratio = img.shape[0] / rsz.shape[0]
    
    return rsz, ratio

def rotate(img, angle, center = None, scale = 1.0):
    (h, w) = img.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rtt = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderValue=(255, 255, 255))

    return rtt

In [226]:
def onMouseExtend(event, x, y, flags, param, img=None, pos_list=None):
    if event == cv2.EVENT_LBUTTONDOWN:
        pos_list.append((x, y))
        cv2.circle(img, pos_list[-1], 5, (0,0,255), -1)
        
class CallbackObject:
    def __init__(self, fn):
        self.__fn = fn
        self.__kwargs = dict()
        
    def update_kwargs(self, **kwargs):
        self.__kwargs.update(kwargs)
        
    def __call__(self, *args, **kwargs):
        return self.__fn(*args, **kwargs, **self.__kwargs)

In [235]:
def cv_crop_image(img, label, height=900, threshold=False, verbose=False):
    rsz, ratio = resize_image(img, height=height)
    if threshold:
        rsz = cv2.cvtColor(rsz, cv2.COLOR_BGR2GRAY)
        rsz = cv2.adaptiveThreshold(rsz, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 0)
        rsz = cv2.cvtColor(rsz, cv2.COLOR_GRAY2BGR)

    image = rsz.copy()
    pos_list = []

    onMouse = CallbackObject(onMouseExtend)
    onMouse.update_kwargs(img=image, pos_list=pos_list)

    rect_rsz = None
    label = label+' ~ Cropping'
    cv2.namedWindow(label)
    cv2.setMouseCallback(label, onMouse)
    while True:
        cv2.imshow(label, image)
        
        key = cv2.waitKey(20) & 0xFF
        if key == 27: # Escape
            # option 1
            pos_list = []
            image = rsz.copy()
            onMouse.update_kwargs(img=image, pos_list=pos_list)
        elif key in (13, 32): # Enter or Space
            if verbose: print(pos_list)
            rect_rsz = np.asarray(pos_list).reshape([-1, 1, 2])
            break
    cv2.destroyAllWindows()
    
    rect_img = ((rect_rsz.astype('float'))*ratio).astype('int')
    x, y, w, h = cv2.boundingRect(rect_img)
    img = img[y:y+h, x:x+w, :]
    
    return img

def cv_rotate_image(img, label, height=900, threshold=False, verbose=False):
    rsz, ratio = resize_image(img, height=height)
    if threshold:
        rsz = cv2.cvtColor(rsz, cv2.COLOR_BGR2GRAY)
        rsz = cv2.adaptiveThreshold(rsz, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21 , 2)
        rsz = cv2.cvtColor(rsz, cv2.COLOR_GRAY2BGR)

    image = rsz.copy()
    pos_list = []

    onMouse = CallbackObject(onMouseExtend)
    onMouse.update_kwargs(img=image, pos_list=pos_list)

    left, right = None, None
    label = label+' ~ Angle'
    cv2.namedWindow(label)
    cv2.setMouseCallback(label, onMouse)
    while True:
        cv2.imshow(label, image)
        
        key = cv2.waitKey(20) & 0xFF
        if key == 27: # Escape
            # option 2 (preferable)
            pos_list.clear()
            image[:,:,:] = rsz
        elif key in (13, 32): # Enter or Space
            if len(pos_list) < 2:
                continue
            if verbose: print(pos_list)
            left, right = pos_list[:2]
            break
    cv2.destroyAllWindows()
    
    angle = np.arctan( (right[1]-left[1])/(right[0]-left[0]) ) * 180 / np.pi
    if verbose: print(angle)
        
    img = rotate(img, angle)
    
    return img

def cv_save_image(img, file_name, label, verbose=False):
    rsz, ratio = resize_image(img, height=800)
    done = False
    
    label = label+' ~ Result'
    cv2.namedWindow(label)

    cv2.imshow(label, rsz)

    key = cv2.waitKey() & 0xFF
    if key == 27: # Escape
        done = False
    elif key in (13, 32): # Enter or Space
        cv2.imwrite(file_name, img)
        done = True
        
    cv2.destroyAllWindows()
    
    return done

In [236]:
def cv2_manual_photo(file_name, iputdir='', oputdir='', label=None):
    oputdir = oputdir or iputdir
    img = read_image(os.path.join(iputdir, file_name), margin=0)

    while True:
        img_proc = cv_crop_image(img, label=label, threshold=True)

        img_proc = cv_rotate_image(img_proc, label=label, threshold=True)

        img_proc = cv_crop_image(img_proc, label=label, threshold=True)
        
        if cv_save_image(img_proc, os.path.join(oputdir, file_name.replace('_orig.png', '.png')), label=label):
            break

In [237]:
iputdir = 'photo_2_crop'

png_name_list = [png for png in os.listdir(iputdir) if png.endswith('_orig.png')]
for i, png_name in enumerate(png_name_list, start=1):
    cv2_manual_photo(png_name, iputdir, label='[{} of {}] {}'.format(i, len(png_name_list), png_name))
        
print("Done!")

Done!
